In [20]:
import numpy as np
import pandas as pd
import math

# Reading the training and testing datasets into the test and train
train_dataset = pd.read_csv("train.csv", header=None)
test_dataset = pd.read_csv("test.csv", header=None)

# Next step is to separate features and labels for training and test sets
features_train = train_dataset.iloc[:, :-1]
labels_train = train_dataset.iloc[:, -1]
features_train_aug = np.column_stack(([1] * features_train.shape[0], features_train))

features_test = test_dataset.iloc[:, :-1]
features_test_aug = np.column_stack(([1] * features_test.shape[0], features_test))
labels_test = test_dataset.iloc[:, -1]

# Let's set parameters according to the question
num_samples, num_features = features_train_aug.shape
weight_vector = np.zeros(num_features)
learning_rate = 1
epochs = 10
feature_matrix = np.array(features_train_aug)
label_vector = np.array(labels_train)
mismatch_count = 0
mod_vector = np.zeros(num_features)

# Prediction function
def make_prediction(sample, weights):
    return 1 if np.dot(weights.T, sample) > 0 else 0

# Voted perceptron function
def voted_weighted_prediction(features, weight_list, count_list):
    predictions = []
    for sample in features:
        score = 0
        for weights, count in zip(weight_list, count_list):
            score += count * np.sign(np.dot(weights, sample))
        predictions.append(1 if score >= 0 else 0)
    return predictions

# Modified training loop with weight vector aggregation
weight_iteration = 0
distinct_weight_vectors = []
count_list = []

# Adjust label values
for i in range(len(label_vector)):
    if label_vector[i] == 0:
        label_vector[i] = -1

weight_iteration = 0
weights_list = [np.zeros_like(feature_matrix)[0]]
count_values = [0]

for epoch in range(epochs):
    for i in range(len(feature_matrix)):
        predicted_label = 1 if np.dot(weights_list[weight_iteration], feature_matrix[i]) > 0 else -1
        if predicted_label == label_vector[i]:
            count_values[weight_iteration] += 1
        else:
            weights_list.append(weights_list[weight_iteration] + learning_rate * label_vector[i] * feature_matrix[i])
            count_values.append(1)
            weight_iteration += 1

distinct_weight_vectors = weights_list
count_list = count_values

# Testing
predicted_labels_test = voted_weighted_prediction(features_test_aug, distinct_weight_vectors, count_list)
test_error_count = 0
for actual, predicted in zip(labels_test, predicted_labels_test):
    if actual != predicted:
        test_error_count += 1

# Display results according to the question
print("Distinct weight vectors are:")
for i, weight_vector in enumerate(distinct_weight_vectors):
    print(f"Weight vector {i + 1}: {weight_vector.tolist()}")
print("And their respective counts:", count_list)
print("Correctly predicted points:", len(features_test_aug) - test_error_count)
print("Average test error:", test_error_count / len(features_test_aug))


Distinct weight vectors are:
Weight vector 1: [0.0, 0.0, 0.0, 0.0, 0.0]
Weight vector 2: [1.0, -1.2667, 2.8183, -2.426, -1.8862]
Weight vector 3: [0.0, -3.4700999999999995, -3.1764, -2.95609, -2.73618]
Weight vector 4: [1.0, -4.8361, -2.9922400000000002, -2.0507, -1.15558]
Weight vector 5: [2.0, -3.386, 0.6144599999999998, -6.1064, -2.75218]
Weight vector 6: [3.0, -8.0625, -5.0491399999999995, 4.8626, -3.0866700000000002]
Weight vector 7: [4.0, -7.94658, -1.8272399999999993, 1.4323999999999995, -5.932370000000001]
Weight vector 8: [3.0, -7.82034, -12.14884, 5.144499999999999, 0.18612999999999946]
Weight vector 9: [4.0, -7.6990799999999995, -11.92537, 4.671229999999999, 1.1563699999999995]
Weight vector 10: [3.0, -11.480879999999999, -9.040769999999998, 2.415429999999999, 1.3137099999999995]
Weight vector 11: [4.0, -11.971689999999999, -6.195569999999998, -1.2281700000000013, -1.7866900000000006]
Weight vector 12: [5.0, -9.777389999999999, -1.6452699999999982, -6.204170000000001, -4.512